In [32]:
import math
import sys
from typing import List
from functools import lru_cache, reduce, cmp_to_key
from itertools import starmap, accumulate, chain, islice, product, tee, pairwise
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
from importlib import reload
reload(arrays)
reload(trees)
import arrays as ar
import trees as tr
import lists as ls
from lists import Node
print()

In [182]:
def permutations(s: str) -> list:
    sols=[]
    n=len(s)
    s = list(s)
    def f(i):
        if i >= n:
            sols.append(''.join(s))
            return
        f(i+1)
        for j in range(i+1, n):
            s[j],s[i] = s[i],s[j]
            f(i+1)
            s[j],s[i] = s[i],s[j]
    f(0)
    return sorted(sols)

In [183]:
permutations('ABC')

['ABC', 'ACB', 'BAC', 'BCA', 'CAB', 'CBA']